## Option 2  pour Creer un model de Classification lite pour des Edges Devices 

In [ ]:
from tflite_model_maker import model_spec
from tflite_model_maker import image_classifier
from tflite_model_maker.config import ExportFormat
from tflite_model_maker.config import QuantizationConfig
from tflite_model_maker.image_classifier import DataLoader
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
import os
import shutil
numbre_of_imgperclass=0

dataset_directory="C:/Users/oboum/Downloads/archive3/archive/asl_alphabet_train/asl_alphabet_train/"
new_dataset="C:/Users/oboum/Downloads/archive3/archive/asl_alphabet_train/new_dataset/"
for dir in os.listdir(dataset_directory):
    for img in os.listdir(dataset_directory+dir):
      if os.path.exists(new_dataset+dir+"/")==False:
         os.makedirs(new_dataset+dir+"/")
      if numbre_of_imgperclass<300:
            shutil.copy(dataset_directory+dir+"/"+img,new_dataset+dir+"/"+img)
            numbre_of_imgperclass=numbre_of_imgperclass+1
      else:
          numbre_of_imgperclass=0
          break
      #print(img)


In [ ]:
#image_path="C:/Users/oboum/Downloads/archive3/archive/asl_alphabet_train/asl_alphabet_train/"
image_path=new_dataset
data = DataLoader.from_folder(image_path)
train_data, test_data = data.split(0.8)

INFO:tensorflow:Load image with size: 8700, num_label: 29, labels: A, B, C, D, E, F, G, H, I, J, K, L, M, N, O, P, Q, R, S, T, U, V, W, X, Y, Z, del, nothing, space.


In [ ]:
model = image_classifier.create(train_data,model_spec='efficientnet_lite3',batch_size=90,epochs=17,shuffle=True)

INFO:tensorflow:Retraining the models...


INFO:tensorflow:Retraining the models...


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hub_keras_layer_v1v2_3 (Hub  (None, 1280)             6992768   
 KerasLayerV1V2)                                                 
                                                                 
 dropout_3 (Dropout)         (None, 1280)              0         
                                                                 
 dense_3 (Dense)             (None, 29)                37149     
                                                                 
Total params: 7,029,917
Trainable params: 37,149
Non-trainable params: 6,992,768
_________________________________________________________________
None
Epoch 1/17
77/77 [==============================] - 239s 3s/step - loss: 2.8575 - accuracy: 0.3258
Epoch 2/17
77/77 [==============================] - 240s 3s/step - loss: 1.9634 - accuracy: 0.7098
Epoch 3/17
77/77 [================

In [ ]:
loss, accuracy = model.evaluate(test_data)

In [ ]:
#model.export(export_dir='.')
model.export(export_dir='./asl_alphabet_train',label_filename='model300_mobilnetv2.tflite')

In [ ]:
## Convert to tflite model
#Our model is all trained up and ready to be used for detecting objects. First, we need to export the model graph (a file that contains information about the architecture and weights) to a TensorFlow Lite-compatible format.
tflite_model = tf.keras.models.load_model('SignLanguageALSClassifier.h5')
converter = tf.lite.TFLiteConverter.from_keras_model_file(tflite_model)
tflite = converter.convert()
PATH_TO_MODEL='SignLanguageALSClassifier_compressed.tflite'
with open(PATH_TO_MODEL, 'wb') as f:
  f.write(tflite)


In [ ]:

#I am having one TensorFlow Keras module "model.h5". I want to generate tflite from it. I am using the below-mentioned code for that. I am using tensorflow version '2.0.0'.

import tensorflow as tf
import numpy as np
from tensorflow import lite

dataset_dir = "C:/Users/oboum/Downloads/archive3/archive/asl_alphabet_train/asl_alphabet_train/"
IMAGE_SIZE = 224
saved_keras_model = "'SignLanguageALSClassifier.h5"

def representative_data_gen():
  dataset_list = tf.data.Dataset.list_files(dataset_dir + '/*')
  for i in range(100):
    image = next(iter(dataset_list))
    image = tf.io.read_file(image)
    image = tf.io.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [IMAGE_SIZE, IMAGE_SIZE])
    image = tf.cast(image / 255., tf.float32)
    image = tf.expand_dims(image, 0)
    yield [image]

converter =  lite.TFLiteConverter.from_keras_model_file(saved_keras_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
# This ensures that if any ops can't be quantized, the converter throws an error
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# These set the input and output tensors to uint8
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8
# And this sets the representative dataset so we can quantize the activations
converter.representative_dataset = representative_data_gen
tflite_model = converter.convert()

with open('efficient_v0_quant.tflite', 'wb') as f:
      f.write(tflite_model)

In [ ]:
## Quantize model
"""I'll use the "TFLiteConverter" module to perform [post-training quantization] on the model.
To quantize the model, we need to provide a representative dataset, which is a set of images that represent what the model will see when deployed in the field. First, we'll create a list of images to include in the representative dataset.
"""
from tensorflow.lite.python.interpreter import Interpreter

PATH_TO_MODEL='mobilenet_model_v3.tflite'
interpreter = Interpreter(model_path=PATH_TO_MODEL) 
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
height = input_details[0]['shape'][1]
width = input_details[0]['shape'][2]
import random

quant_image_list=x_train
def representative_data_gen():
  dataset_list = quant_image_list
  quant_num = 300
  for i in range(quant_num):
    pick_me = random.choice(dataset_list)
    image = tf.io.read_file(pick_me)

    if pick_me.endswith('.jpg') or pick_me.endswith('.JPG'):
      image = tf.io.decode_jpeg(image, channels=3)
    elif pick_me.endswith('.png'):
      image = tf.io.decode_png(image, channels=3)
    elif pick_me.endswith('.bmp'):
      image = tf.io.decode_bmp(image, channels=3)

    image = tf.image.resize(image, [width, height]) 
    image = tf.cast(image / 255., tf.float32)
    image = tf.expand_dims(image, 0)
    yield [image]


# Load the saved model
model = tf.keras.models.load_model('saved_model_mobileNet')

# Convert the model to TFLite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
# This enables quantization
converter.optimizations = [tf.lite.Optimize.DEFAULT]
# This sets the representative dataset for quantization
converter.representative_dataset = representative_data_gen

# This ensures that if any ops can't be quantized, the converter throws an error
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# For full integer quantization, though supported types defaults to int8 only, we explicitly declare it for clarity.
converter.target_spec.supported_types = [tf.int8]
# These set the input tensors to uint8 and output tensors to float32
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.float32

tflite_model = converter.convert()


# Save the TFLite model to a file
with open('quant_mobilenet_ASLv3.tflite', 'wb') as f:
    f.write(tflite_model)